In [0]:
%sql
CREATE TABLE IF NOT EXISTS workspace.gold_serving.fact_results
USING DELTA
AS
SELECT
  time,
  fk,
  samp_id,
  as_id,
  Type,
  comp,
  unit,
  det_id,
  result,
  number_of_averaged_samples
FROM workspace.gold_staging.stg_results;

MERGE INTO workspace.gold_serving.fact_results AS target
USING (
  SELECT * FROM (
    SELECT
      time,
      fk,
      samp_id,
      as_id,
      Type,
      comp,
      unit,
      det_id,
      result,
      number_of_averaged_samples,
      ROW_NUMBER() OVER (
        PARTITION BY time, fk, samp_id
        ORDER BY time DESC
      ) AS rn
    FROM workspace.gold_staging.stg_results
  ) WHERE rn = 1
) AS source
ON target.fk = source.fk
AND target.time = source.time

WHEN MATCHED THEN
  UPDATE SET
    target.as_id = source.as_id,
    target.samp_id = source.samp_id,
    target.Type = source.Type,
    target.comp = source.comp,
    target.unit = source.unit,
    target.det_id = source.det_id,
    target.result = source.result,
    target.number_of_averaged_samples = source.number_of_averaged_samples
WHEN NOT MATCHED THEN
  INSERT (
    time,
    fk,
    samp_id,
    as_id,
    Type,
    comp,
    unit,
    det_id,
    result,
    number_of_averaged_samples
  )
  VALUES (
    source.time,
    source.fk,
    source.samp_id,
    source.as_id,
    source.Type,
    source.comp,
    source.unit,
    source.det_id,
    source.result,
    source.number_of_averaged_samples
  );
